In [1]:
from __future__ import unicode_literals
import re
import nltk
from Politweet import get_tweets, get_transcript
import ratings
from sentiment import plus_df, minus_df
import pandas as pd
pd.set_option('display.max_colwidth', 1200)

In [2]:
tweets = get_tweets("./datasets/tweets.tsv")
transcript = get_transcript("./datasets/transcript.csv")
todo = set(tweets.index)


In [5]:
tweets.to_pickle('./datasets/tweets.pickle')

In [3]:
# polarity = polarity_train(tweets)
# polarity_minus = set(i for i,_ in classify(minus_df(tweets), polarity))
# polarity_plus = set(i for i,_ in classify(plus_df(tweets), polarity))
# polarities = tweets.reindex(polarity_minus | polarity_plus).sort()
# print len(polarities), "- polarity"
# todo -= set(polarities.index)

In [12]:
others1 = ratings.all(tweets, ratings.OTHER)
others2= ratings.all(tweets, ratings.POSITIVE)
others3 = ratings.all(tweets, ratings.NEGATIVE)
others4 = ratings.all(tweets, ratings.MIXED)
len(tweets)

3238

In [9]:
print len(todo), "- untagged"

3087 - untagged


In [10]:
left = tweets.reindex(todo).sort()

In [11]:
left[["content","tokens", "clean"]][:2]

,content,tokens,clean
tweet.id,,,
936466790,Jim Lehrer just directed the debate audience ... 30 seconds ... #tweetdebate,"[jim, lehrer, direct, debate, audience, second, tweetdebate]","[{u'lemma': u'jim', u'token': u'jim', u'pos': u'NN'}, {u'lemma': u'lehrer', u'token': u'lehrer', u'pos': u'NN'}, {u'lemma': u'direct', u'token': u'directed', u'pos': u'VBN'}, {u'lemma': u'debate', u'token': u'debate', u'pos': u'NN'}, {u'lemma': u'audience', u'token': u'audience', u'pos': u'NN'}, {u'lemma': u'', u'token': u'', u'pos': u''}, {u'lemma': u'30', u'token': u'30', u'pos': u'CD'}, {u'lemma': u'second', u'token': u'seconds', u'pos': u'NNS'}, {u'lemma': u'', u'token': u'', u'pos': u''}, {u'lemma': u'', u'token': u'', u'pos': u''}, {u'lemma': u'tweetdebate', u'token': u'tweetdebate', u'pos': u'NN'}]"
936466992,Here we go. #tweetdebate,[tweetdebate],"[{u'lemma': u'go', u'token': u'go', u'pos': u'VBP'}, {u'lemma': u'', u'token': u'', u'pos': u''}, {u'lemma': u'', u'token': u'', u'pos': u''}, {u'lemma': u'tweetdebate', u'token': u'tweetdebate', u'pos': u'NN'}]"


In [12]:
for i, tweet in tweets[:3].iterrows():
    print [token for token in tweet["tokens"]]
    
tokens = [" ".join(tweet["tokens"]) for i,tweet in tweets.iterrows()]
tokens[:1]

[u'watch', u'tweetdebate', u'drink', u'wait', u'start', u'cringing', u'mccain', u'blunder']
[u'ahg3', u'michdot', u'yeah', u'slime', u'actually', u'second', u'choice', u'say', u'first', u'one', u'okay', u'roll']
[u'prepare', u'heart', u'attack', u'tweetdebate']


[u'watch tweetdebate drink wait start cringing mccain blunder']

In [13]:
# from collections import Counter
# from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# count = Counter([token for i, tweet in tweets.iterrows() for token in tweet["tokens"]])

# vectorizer = TfidfVectorizer(lowercase=False)
# tfidf_model = vectorizer.fit_transform(tokens)
# feature_names = vectorizer.get_feature_names()
# df = pd.DataFrame([
#         (feature_names[col], tfidf_model[0, col])
#         for col in tfidf_model.nonzero()[1]])
# df.sort(1, ascending=False)

In [14]:
# for col in tfidf_model.nonzero()[1]:
#     print tfidf_model[0, col]
#     print (feature_names[col], tfidf_model[0, col])

In [15]:
# tweets[tweets["content"].str.contains("zoom")][['content']]

In [19]:
import json
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk import sent_tokenize, word_tokenize, FreqDist, WordNetLemmatizer
from nltk.corpus import stopwords

neg = [
    (t['clean'], 'neg')
    for i,t in ratings.all(tweets, ratings.NEGATIVE).iterrows()]

pos = [
    (t['clean'], 'pos')
    for i,t in ratings.all(tweets, ratings.POSITIVE).iterrows()]

train, test = train_test_split(
    pos + neg, 
    test_size = .2, 
    random_state = 10)

def featurize(tweet):
#     # tokenize into words
#     tokens = [word for sent in sent_tokenize(tweet) for word in word_tokenize(sent)]

#     # remove stopwords
#     stop = stopwords.words('english')
#     tokens = [token for token in tokens if token not in stop]

#     # remove words less than three letters
#     tokens = [word for word in tokens if len(word) >= 3]

#     # lower capitalization
#     tokens = [word.lower() for word in tokens]

#     # lemmatize
#     lmtzr = WordNetLemmatizer()
#     tokens = [lmtzr.lemmatize(word) for word in tokens]
    tokens = [token['lemma'] for token in tweet]
    return tokens

def run_classifier(train, test, vctrzr, clsfr):
    # vectorize with above featurize() function and also vectorize test set with training fit
    X_train = vctrzr.fit_transform([i[0] for i in train])
    X_test = vctrzr.transform([i[0] for i in test])

    # fit the classifier with training data
    clsfr.fit(X_train, [i[1] for i in train])
    
    # grab accuracy
    scr = clsfr.score(X_test, [i[1] for i in test])

    # grab important features
    imp_features = sorted(zip(clsfr.coef_[0], vctrzr.get_feature_names()))
    
    return scr, imp_features

In [27]:
vectorizer = CountVectorizer(tokenizer = featurize, binary = True, lowercase=False)
classifier = MultinomialNB()

score, imp_features = run_classifier(train, test, vectorizer, classifier)
score, imp_features[0:40], imp_features[-21:-1]

1: 3 debate 2 1 debate08 mccain current tweetdebate obama 


In [30]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2), tokenizer = featurize, lowercase=False)
classifier = LinearSVC()

score, imp_features = run_classifier(train, test, vectorizer, classifier)
score, imp_features[0:20]
score, imp_features[0:20], imp_features[-21:-1]

(0.80434782608695654,
 [(-1.5118572777957981, u'mccain'),
  (-1.2997493986362076, u'nt'),
  (-1.005156629495646, u'stop'),
  (-0.71799039922568453, u'talk'),
  (-0.71326772291679796, u'would'),
  (-0.69893576769163457, u'get'),
  (-0.6911008221866769, u'obama need'),
  (-0.67873458646308049, u'different'),
  (-0.67812866856344434, u'republican'),
  (-0.66136567813024927, u'much'),
  (-0.65597160624328954, u'miss'),
  (-0.64716853403401231, u'bracelet'),
  (-0.64276486143685407, u'lose'),
  (-0.61415475358861993, u'old'),
  (-0.60079648447841372, u'palin'),
  (-0.59643126624004761, u'yet'),
  (-0.5638306654817794, u'claim'),
  (-0.56029515400711249, u'debate08 mccain'),
  (-0.55158729605147994, u'shit'),
  (-0.55158729605147994, u'shit ')],
 [(0.74286631318042384, u'see'),
  (0.76177326576147919, u'obama 3'),
  (0.77700408089090667, u'win '),
  (0.78061688767219239, u'great'),
  (0.78162383498319921, u'hold'),
  (0.78281442093158682, u'twitter'),
  (0.79774186068497355, u'debate '),
  (